# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
DATA_FOLDER = 'Data/'
import pandas as pd
import numpy as np
import glob
import matplotlib
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

## Solution

### Explanation
By looking at the data files from those 3 countries, we can see that there 3 types of new cases (confirmed, probable and suspected) that are represented every time by 3 different fields, and 1 field that corresponds to the number of new deaths. For those 3 countries, the data are represented in slightly different ways, using different terminologies. It thus makes sense to go over those 3 countries one after the other, clean, filter and uniformize the data, compute what we need (which is the average number of new cases and new deaths per day for each month) and then put those 3 countries and what we've computed together in a final DataFrame.

### Corresponding code

In [2]:
# Constants
EBOLA_FOLDER = DATA_FOLDER + 'ebola/'
CONFIRMED_NEW_CASES = 'New confirmed cases'
PROBABLE_NEW_CASES = 'New probable cases'
SUSPECTED_NEW_CASES = 'New suspected cases'
NEW_DEATHS = 'New deaths'
AVG_CONFIRMED_NEW_CASES = 'New confirmed cases (avg per day)'
AVG_PROBABLE_NEW_CASES = 'New probable cases (avg per day)'
AVG_SUSPECTED_NEW_CASES = 'New suspected cases (avg per day)'
AVG_NEW_DEATHS = 'New deaths (avg per day)'

In [3]:
# Utilitary functions
def read_folder(date, folder):
    df = pd.DataFrame()
    for file in folder:
        tmp_df = pd.read_csv(file,header=0,index_col=None,parse_dates=[date])
        df = pd.concat([df, tmp_df])
    return df

def renamed_df(df):
    # We rename the columns for the intermediate output
    return df.rename(columns={
        CONFIRMED_NEW_CASES: AVG_CONFIRMED_NEW_CASES,
        PROBABLE_NEW_CASES: AVG_PROBABLE_NEW_CASES,
        SUSPECTED_NEW_CASES: AVG_SUSPECTED_NEW_CASES,
        NEW_DEATHS: AVG_NEW_DEATHS})

### DataFrames creation
For each country, we first have to read every CSV file and put their content into one big DataFrame. This will allow us to then work on the DataFrame (filter data, clean data, aggregate what we need, ...).

In [4]:
file_gui = glob.glob(EBOLA_FOLDER + "guinea_data/*.csv")
file_lib = glob.glob(EBOLA_FOLDER + "liberia_data/*.csv")
file_sie = glob.glob(EBOLA_FOLDER + "sl_data/*.csv")

ebola_gui = read_folder('Date', file_gui)
ebola_lib = read_folder('Date', file_lib)
ebola_sie = read_folder('date', file_sie)

### Guinea
Here we want to extract the average per day for every month for all 3 types of new cases and for new deaths in Guinea. What we have to do is to filter the data for each case (we have 4 cases) we want to have, get each time the corresponding number of new cases or deaths, and merge those 4 DataFrames into 1 by the date. Once this is done, we can extract the month and year from the date, group by the month and year, and compute the average as an aggregation.

In [5]:
def get_guinea_specific_df(description, new_col_name):
    res = ebola_gui.where(lambda row: row['Description'] == description)
    df = res[pd.notnull(res['Description'])]
    df = df[['Totals', 'Date']]
    df = df.rename(columns = {'Totals': new_col_name})
    return df
    
new_cases_confirmed = get_guinea_specific_df('New cases of confirmed', CONFIRMED_NEW_CASES)
new_cases_probable = get_guinea_specific_df('New cases of probables', PROBABLE_NEW_CASES)
new_cases_suspect = get_guinea_specific_df('New cases of suspects', SUSPECTED_NEW_CASES)
# Note: This makes the first file (2014-08-04) disappear because it is registered under "New deaths registered today"
# and not "New deaths registered" like the others.
new_deaths_registered = get_guinea_specific_df('New deaths registered', NEW_DEATHS)

# Put all things together
guinea_df = new_cases_confirmed.merge(new_cases_probable).merge(new_cases_suspect).merge(new_deaths_registered)
guinea_df['Country'] = 'Guinea'
guinea_df['Date'] = guinea_df.Date.dt.strftime("%B %Y")
guinea_df = guinea_df.apply(pd.to_numeric, errors='ignore')

# Group by to compute the average per day for each month
guinea_df = guinea_df.groupby(['Date', 'Country']).mean()

renamed_df(guinea_df)

,,New confirmed cases (avg per day),New probable cases (avg per day),New suspected cases (avg per day),New deaths (avg per day)
Date,Country,,,,
August 2014,Guinea,14.5,2.666667,13.5000,3.7500
October 2014,Guinea,6.0,0.000000,28.0000,15.0000
September 2014,Guinea,13.0,1.187500,5.4375,3.5625


### Liberia
The idea here is similar. The big difference is that there are numbers that seem wrongly reported in December (you can see huge numbers, bigger than 1000), so we want to filter them out so that they don't mess up our data. To do that we choose a threshold (500) and if we find a value bigger than the threshold, we remove the row because it is very likely that this is a wrongly reported value.

In [7]:
def get_liberia_specific_df(variable, new_col_name):
    res = ebola_lib.where(lambda row: row['Variable'] == variable)
    df = res[pd.notnull(res['Variable'])]
    df = df[pd.isnull(df['National']) | (df['National'] < 500.0)] # Remove wrongly reported data, but keep NaN for now
    df = df[['National', 'Date']]
    df = df.rename(columns = {'National': new_col_name})
    return df

new_cases_confirmed = get_liberia_specific_df('New case/s (confirmed)', CONFIRMED_NEW_CASES)
new_cases_probable = get_liberia_specific_df('New Case/s (Probable)', PROBABLE_NEW_CASES)
new_cases_suspect = get_liberia_specific_df('New Case/s (Suspected)', SUSPECTED_NEW_CASES)
new_deaths_registered = get_liberia_specific_df('Newly reported deaths', NEW_DEATHS)

# Put all things together
liberia_df = new_cases_confirmed.merge(new_cases_probable, on='Date').merge(new_cases_suspect, on='Date').merge(new_deaths_registered, on='Date')
liberia_df['Country'] = 'Liberia'
liberia_df['Date'] = liberia_df.Date.dt.strftime("%B %Y")
liberia_df = liberia_df.apply(pd.to_numeric, errors='ignore')

# Group by to compute the average per day for each month
liberia_df = liberia_df.groupby(['Date', 'Country']).mean()

renamed_df(liberia_df)

,,New confirmed cases (avg per day),New probable cases (avg per day),New suspected cases (avg per day),New deaths (avg per day)
Date,Country,,,,
August 2014,Liberia,5.444444,19.777778,12.000000,23.222222
December 2014,Liberia,5.000000,9.500000,21.500000,NaN
July 2014,Liberia,1.818182,3.727273,3.000000,4.272727
June 2014,Liberia,2.142857,1.142857,2.428571,2.000000
November 2014,Liberia,7.800000,8.076923,19.461538,15.538462
October 2014,Liberia,1.619048,17.760000,26.440000,28.040000
September 2014,Liberia,6.166667,29.333333,28.333333,37.608696


### Sierra Leone
Again, the idea is the same. But this time, we don't have a number that gives us exactly the new number of deaths. So instead, we are using 'etc_new_deaths' for computing the number of deaths per day; 'etc' means 'Ebola Treatment Center', so those seem to be the deaths happening in the treatment center.
Another solution would have been to use 'death_confirmed', 'death_suspected' and 'death_probable', but those values are cumulative, so we would have had to compute the differences every day, and be sure that we have data every day to have accurate values, but this is not the case here.

In [8]:
def get_sl_specific_df(variable, new_col_name):
    res = ebola_sie.where(lambda row: row['variable'] == variable)
    df = res[pd.notnull(res['variable'])]
    df = df[['National', 'date']]
    df = df.rename(columns = {'National': new_col_name})
    return df

new_cases_confirmed = get_sl_specific_df('new_confirmed', CONFIRMED_NEW_CASES)
new_cases_probable = get_sl_specific_df('new_probable', PROBABLE_NEW_CASES)
new_cases_suspect = get_sl_specific_df('new_suspected', SUSPECTED_NEW_CASES)
new_deaths_registered = get_sl_specific_df('etc_new_deaths', NEW_DEATHS)

# Put all things together
sl_df = new_cases_confirmed.merge(new_cases_probable, on='date').merge(new_cases_suspect, on='date').merge(new_deaths_registered, on='date')
sl_df['Country'] = 'Sierra Leone'
sl_df['date'] = sl_df.date.dt.strftime("%B %Y")
sl_df = sl_df.apply(pd.to_numeric, errors='ignore')

# Group by to compute the average per day for each month
sl_df = sl_df.groupby(['date', 'Country']).mean()

renamed_df(sl_df)

,,New confirmed cases (avg per day),New probable cases (avg per day),New suspected cases (avg per day),New deaths (avg per day)
date,Country,,,,
August 2014,Sierra Leone,20.611111,1.277778,6.055556,NaN
December 2014,Sierra Leone,54.333333,0.000000,14.000000,5.500000
November 2014,Sierra Leone,66.210526,0.000000,23.000000,2.400000
October 2014,Sierra Leone,56.714286,0.888889,13.851852,5.210526
September 2014,Sierra Leone,36.962963,0.000000,7.000000,2.666667


### Put everything together
Finally, we just want to concatenate everything together and rename the different columns accordingly.

In [9]:
# Concatenate the values for the 3 countries and rename columns accordingly
final_df = pd.concat([guinea_df, liberia_df, sl_df])
final_df = renamed_df(final_df)
final_df = final_df.fillna('unknown')
final_df

,,New confirmed cases (avg per day),New probable cases (avg per day),New suspected cases (avg per day),New deaths (avg per day)
Date,Country,,,,
August 2014,Guinea,14.500000,2.666667,13.500000,3.75
October 2014,Guinea,6.000000,0.000000,28.000000,15
September 2014,Guinea,13.000000,1.187500,5.437500,3.5625
August 2014,Liberia,5.444444,19.777778,12.000000,23.2222
December 2014,Liberia,5.000000,9.500000,21.500000,unknown
July 2014,Liberia,1.818182,3.727273,3.000000,4.27273
June 2014,Liberia,2.142857,1.142857,2.428571,2
November 2014,Liberia,7.800000,8.076923,19.461538,15.5385
October 2014,Liberia,1.619048,17.760000,26.440000,28.04


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

## Solution

### Explanation

Here we have 9 spreedsheets containing microbiome data. This data can be seen as a single table containing 2 columns, the `NAME` and the `COUNT`. Thus, we simply read those files and concatenate them into a single dataframe representing our table with 2 columns.

Given that the file does not contain headers, we ignore the first row (which is not a header) and give a name to the columns with the `header=None` and `names=['NAME', 'COUNT']` parameters.

We also add a new `BARCODE` column to the dataframe, which is the name corresponding to the column name in the metadata file. We give it the same name in order to have normalized dataframes that can be merged on columns with the same name.

Finally, we read the metadata file into a dataframe that we merge with our initial dataframe, we replace NA values with the `unknown` value and place an index on `[BARCODE, NAME]` in order to have unique indexes

### Corresponding Code

In [10]:
MICROBIOME_FOLDER = DATA_FOLDER + 'microbiome/'
mid_files = glob.glob(MICROBIOME_FOLDER + "MID*.xls")

In [11]:
# Initial empty dataframe
mid = pd.DataFrame()
for idx in range(1, len(mid_files)+1):
    file_name = 'MID' + str(idx)
    # We give appropriate names to colummns
    tmp_mid = pd.read_excel(MICROBIOME_FOLDER+file_name+'.xls', names=['NAME', 'COUNT'], header=None)
    # We add a column "BARCODE" corresponding to the name of the file being read
    tmp_mid['BARCODE'] = file_name
    # We concatenate the values to create a single dataframe
    mid = pd.concat([mid, tmp_mid])

# We read the metadata file and merge it to the MID dataframe where columns have the same name
metadata = pd.read_excel(MICROBIOME_FOLDER + 'metadata.xls')
# We create an unique index on BARCODE and TAXON and replace NA values by 'unknown'
final_df = pd.merge(mid, metadata).fillna('unknown').set_index(['BARCODE', 'NAME'])
final_df

COUNT  \
BARCODE NAME                                                        
MID1    Archaea "Crenarchaeota" Thermoprotei Desulfuroc...      7   
        Archaea "Crenarchaeota" Thermoprotei Desulfuroc...      2   
        Archaea "Crenarchaeota" Thermoprotei Sulfolobal...      3   
        Archaea "Crenarchaeota" Thermoprotei Thermoprot...      3   
        Archaea "Euryarchaeota" "Methanomicrobia" Metha...      7   
        Archaea "Euryarchaeota" "Methanomicrobia" Metha...      1   
        Archaea "Euryarchaeota" "Methanomicrobia" Metha...      1   
        Archaea "Euryarchaeota" Archaeoglobi Archaeoglo...      1   
        Archaea "Euryarchaeota" Archaeoglobi Archaeoglo...      1   
        Archaea "Euryarchaeota" Halobacteria Halobacter...      1   
        Archaea "Euryarchaeota" Halobacteria Halobacter...      4   
        Archaea "Euryarchaeota" Halobacteria Halobacter...      1   
        Archaea "Euryarchaeota" Halobacteria Halobacter...      2   
        Archaea "Euryarchaeota" Halobacteria Halobacter...      4   
        Archaea "Euryarchaeota" Methanococci Methanococ...      1   
        Archaea "Euryarchaeota" Methanopyri Methanopyra...     12   
        Archaea "Euryarchaeota" Thermoplasmata Thermopl...      2   
        Bacteria "Actinobacteria" Actinobacteria Actino...      2   
        Bacteria "Actinobacteria" Actinobacteria Actino...      1   
        Bacteria "Actinobacteria" Actinobacteria Actino...      1   
        Bacteria "Actinobacteria" Actinobacteria Actino...      1   
        Bacteria "Actinobacteria" Actinobacteria Actino...      2   
        Bacteria "Actinobacteria" Actinobacteria Actino...     53   
        Bacteria "Actinobacteria" Actinobacteria Actino...      1   
        Bacteria "Actinobacteria" Actinobacteria Actino...      2   
        Bacteria "Actinobacteria" Actinobacteria Actino...      3   
        Bacteria "Actinobacteria" Actinobacteria Actino...      1   
        Bacteria "Actinobacteria" Actinobacteria Actino...     99   
        Bacteria "Actinobacteria" Actinobacteria Actino...     17   
        Bacteria "Actinobacteria" Actinobacteria Actino...      2   
...                                                           ...   
MID9    Bacteria "Proteobacteria" Gammaproteobacteria O...     10   
        Bacteria "Proteobacteria" Gammaproteobacteria O...      1   
        Bacteria "Proteobacteria" Gammaproteobacteria O...      1   
        Bacteria "Proteobacteria" Gammaproteobacteria P...      1   
        Bacteria "Proteobacteria" Gammaproteobacteria P...     12   
        Bacteria "Proteobacteria" Gammaproteobacteria P...      4   
        Bacteria "Proteobacteria" Gammaproteobacteria P...    805   
        Bacteria "Proteobacteria" Gammaproteobacteria P...    862   
        Bacteria "Proteobacteria" Gammaproteobacteria P...     12   
        Bacteria "Proteobacteria" Gammaproteobacteria P...   3534   
        Bacteria "Proteobacteria" Gammaproteobacteria T...      4   
        Bacteria "Proteobacteria" Gammaproteobacteria T...      1   
        Bacteria "Proteobacteria" Gammaproteobacteria T...     25   
        Bacteria "Proteobacteria" Gammaproteobacteria T...      2   
        Bacteria "Proteobacteria" Gammaproteobacteria X...      3   
        Bacteria "Proteobacteria" Gammaproteobacteria X...      2   
        Bacteria "Proteobacteria" Gammaproteobacteria X...      1   
        Bacteria "Proteobacteria" Gammaproteobacteria X...      3   
        Bacteria "Proteobacteria" Gammaproteobacteria X...      2   
        Bacteria "Proteobacteria" Gammaproteobacteria X...      1   
        Bacteria "Synergistetes" Synergistia Synergista...      3   
        Bacteria "Tenericutes" Mollicutes Anaeroplasmat...      1   
        Bacteria "Tenericutes" Mollicutes Haloplasmatal...     11   
        Bacteria "Thermodesulfobacteria" Thermodesulfob...      1   
        Bacteria "Thermodesulfobacteria" Thermodesulfob...      1   
        Bacteria "Thermotogae" Thermotogae Thermotogale...

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
titanic = pd.read_excel('Data/titanic.xls')
titanic

### 1) Explanation

For each attribute we will display the underlying pandas type and the differents values its takes in the files. This will allow us to determinate which attribute couldbe represented by a Categorical type and transorm it accordingly

In [ ]:
for c in titanic.columns:
    print("%s is of type %s" % (c,titanic[c].dtype))
    print(titanic[c].unique())
    print("\n ------------------------------\n")

From above output we can see that travel class, surviving, sex and embarked have some limited possibilities,
which make them good candidate for categorical type

In [ ]:
#change travel class to categorical type with more significant label
titanic['pclass'] = titanic['pclass'].apply(lambda x: {1:'1st',2:'2nd',3:'3rd'}[x]).astype('category')
#change surviving attribute to categorical type
titanic['survived'] = titanic['survived'].astype('category')
#change sex attribute to categorical type
titanic['sex'] = titanic['sex'].astype('category')
#change embarked attribute to categorical type with more significant label
ports = {'C':'Cherbourg','Q':'Queenstown','S':'Southampton'}
keys = ['C','Q','S']
titanic['embarked'] = titanic['embarked'].apply(lambda x : ports.get(x,ports[keys[randint(0,2)]]))
titanic.pclass.dtype

### 2) Explanation
Histogram plot for travel class, embarkation port, sex and age by decade.
Using the values in the travel class/embarked/sex attribute, we count them and show a plot. For the age we use bins of size 10 so that we have decade intervals.

In [ ]:
hist_pclass = titanic['pclass']
hist_pclass.value_counts().plot.bar()

In [ ]:
hist_emb = titanic['embarked'].value_counts().plot.bar()

In [ ]:
hist_sex = titanic['sex'].value_counts().plot.bar()

In [ ]:
hist_age = titanic['age'].plot.hist(bins=10)

### 3) Explanation
Proportion of passengers by cabin floor.
Using the first letter of the cabin attribute we know on which floor the passenger was sleeping. Again we then count the occurence of each floor and plot it in a pie chart (one with the missing data, one without).

In [ ]:
# We assume that ine value 'A10' the floor is 'A'
# First we take the cabin columns as string, map them on the first letter and the
cabin_plot = titanic['cabin'].astype(str).apply(lambda x : 'unkown' if x=='nan' else x[0]).copy()
cabin_plot.value_counts().plot.pie(autopct='%.2f',figsize=(8, 8))

Since there is so much unkown we plot also the pie chart without them

In [ ]:
# Same values but without the unkown
cabin_plot = titanic['cabin'].dropna().astype(str).apply(lambda x : x[0]).copy()
cabin_plot.value_counts().plot.pie(autopct='%.2f',figsize=(8, 8))

### 4) Explanation
Proportion of the passengers that survived for each travel class

First we get ride of the unecessary data. For this one we use only the travel class and the survived data. We then compute the data of people that died (the inverse of the surviving one) and use those values to plot a pie chart for each travel class.


In [ ]:
# Filter needed columns
survived_data = titanic[['pclass','survived']].copy()
# Created columns for people who died
survived_data['died'] = 1 - survived_data['survived'].astype('int')
# Convert survived column to int type
survived_data['survived'] = survived_data['survived'].astype('int')
# Group by travel class on a sum and transpose (for ploting to works)
survived_plot = survived_data.groupby('pclass').sum().transpose()
# Subplot data on 3 pie charts
survived_plot.plot.pie(subplots=True,autopct='%.2f',figsize=(12, 4))

### 5)Explanation
Proportion of the passengers that survived by travel class and sex.

We just need to use the data of travel class, sex and survived data and plot it.

In [ ]:
#filter needed columns
survived_sex_data = titanic[['pclass','sex','survived']].copy()
#convert survived categorical type to int
survived_sex_data['survived'] = survived_sex_data['survived'].astype('int')
#group by class of pairs and plot (we assume seaborn is allowed since it was used in the intro to Pandas I)
sns.barplot(x='pclass',y='survived',hue='sex',data=survived_sex_data)


### 6) Explanation
Survival proportions by age category, travel class and sex

We first sort the data by the age to be able to compute the median. We then create new data age category and label people under the median as 'young' and the others as 'old'. We also add a column 'acc' which contains only 1s. This will be used to have the total number of person so we will be able to compute the proportion survived/people. Finally we group the date by category age, travel class and sex and compute the proportion of people that survived.

In [ ]:
# Filter needed columns
prop_data = titanic[['pclass','survived','age','sex']].copy().sort_values(['age'])
#drop nan values
prop_data = prop_data[np.isfinite(prop_data['age'])]
# Compute median
median_val = np.median(prop_data.age.dropna())
print("median : %s" % median_val)
# Create new age category
prop_data['age_cat'] = prop_data.age.apply(lambda x : 'young' if x < median_val else 'old')
# Create a accumulator columns to compute proportion
prop_data['acc'] = 1
#convert survived to string
prop_data['survived'] = prop_data['survived'].astype('int')
# Group by new class
final_data = prop_data[['age_cat','pclass','sex','survived','acc']].groupby(['age_cat','pclass','sex']).sum()
final_data['proportion_survived'] = final_data['survived'] / final_data['acc']
final_data[['proportion_survived']]